# ハンズオンの流れ

本日のハンズオンの流れは以下のようになります。

1. Flow Based Programmingの説明(本章)と簡単な利用方法(2,3章)
1. REINVENT4の転移学習を使ってEGFR阻害剤を生成するような生成モデルを作成。その後生成モデルに仮想化合物群を生成させる(4章)。
1. Gypsum-DLでSBVSの前処理をおこなう(5章)
1. AutoDock VinaでEGFRのキナーゼドメインに対しSBVSを実行し、結果をpymolで確認する(6章)
1. 前処理とSBVSはmaizeというflow based programmingツールを使ってワークフロー化します(6章)


## Flow Based Programming (FBP)

フローベースプログラミング（FBP）は、1970年代にIBMに勤務していたJ Paul Morrisonによって考案された。

フローベースのプログラミングでは、アプリケーションを、プロセスが「ポート」によってアクセスするコネクションを介して通信するプロセス（「ブラックボックス」）の集合と見なす。プロセスはコンポーネントのインスタンスであり、他のプロセスと同時に実行される。この表現は、プロセスをノード、コネクションをエッジとする有向グラフである。アプリケーション開発者にとって、コンポーネントはブラックボックスとして扱われるのが普通であるが、開発者が新しいコンポーネントを作成する必要がある場合（通常は、従来のテキスト・プログラミング言語を使用する）、またはコンポーネントがサブグラフとして実装されている場合は、ホワイトボックス化することができる。後者の場合、ユーザーはグラフの異なるレベル間を移動することができる。

FBPの一般的なアプローチは、プログラムを、一連の接続されたプロセスを流れる一連のストリームとサブストリームとして概念化することである。プロセス間の通信を、インフォメーション・パケット（「受動的な」データ）のストリームを使用するように制限することで、並列性と並行性が可能になる。インフォメーション・パケットは、一度に1つのプロセスによってのみ（直接的または間接的に）「所有」されるというルールによって、共有メモリの問題が回避される。

主な動機は、コードの再利用、テスト容易性、並行性、保守性である。

DeepL.com（無料版）で翻訳しました。

### FBP パッケージの例

- Luigi https://github.com/spotify/luigi
- AirFlow https://airflow.apache.org/
- maize https://github.com/MolecularAI/maize

## maize, maize-conbribについて

- maizeはアストラゼネカ社が開発公開しているFBPパッケージです。今回これをチュートリアルとして選んだ理由は、ケモインフォマティクス関連のタスクもカバーできるからです。
もちろん、Luigi, AirFlowなどを利用してもケモインフォマティクスパイプラインの構築はできますが、ケモインフォも部分をフルスクラッチで準備する必要があります。
Reinventなど多くのAI創薬のパッケージとの親和性を加味して設計されたMaizeはまだ開発段階でありドキュメントも不足していますが、なかなか面白いパッケージだと思い今回チュートリアルのお題に選定しました。

- maizeに注目した理由は、上記のようにケモインフォのWFをカバーするのみならず、設定ファイルを指定することで、外部の環境をパイプラインに組み込めることもあります。

- 次のチュートリアルにて設定をしますが、maize.tomlというファイルに各種の設定を記載し読み込ませることでCondaに構築したmaizeという環境外のプログラムも利用することができます。

- 今回利用するREINVENTやGYPSUM-DLはそれぞれ個別のプログラムとして開発されていることから、プログラムの依存関係もそれぞれです。このようなプログラム軍を同じ環境下にインストールするとコンフリクトするリスクがありメンテナンスが非常に煩雑になります。maize.tomlはそのようなコンフリクトからの開放をしてくれます。

- さらにGromacs　Schrodinger VINA　AutoDockVina-GPUなどもこの仕組みを利用することで利用できます。

- 非常に拡張性のあるプログラムの一つと言えるかと思います。

- 日本語でのドキュメントはなく、また公式ドキュメントも全機能を網羅していない開発中のパッケージです。私自身Maizeの詳細を把握していませんが、FBPを知るきっかけになれば幸いです。


## 今回チュートリアルに利用するmaize.tomlの概要
- パスの書き換えは必要になりますが今回利用するTOMLの構成は基本的に以下のようになります。

```bash
# maize 内のVina Nodeに関する設定項目です。
# vinaをコールするためのPATHの指定をしています。
[vina]
python = "/home/iwatobipen/miniforge3/envs/maize/bin/python"
commands.vina = "/opt/vina/bin/vina"

# maize 内のGypsum Nodeに関する設定項目です。
# Gypsum内のプログラムをコールするためのPATHの指定をしています。
[gypsum]
scripts.gypsum.interpreter = "/home/iwatobipen/miniforge3/envs/gypsum/bin/python"
scripts.gypsum.location = "/home/iwatobipen/dev/gypsum_dl/run_gypsum_dl.py"

```

- maize.tomlを書き換えて、ファイルを置いた先をXDG_CONFIG_HOMEという環境変数に設定します。これを設定しておくことで後半のWorkflowのときにConfigをしないで良くなります。
- Linuxの場合デフォルトで~/.configがXDG_CONFIG_HOMEになるのでそこにmaize.tomlをおいていただいてもOKです。

```bash
$ export XDG_CONFIG_HOME='{maize.tomlがある場所}':$XDG_CONFIG_HOME
```

## REINVENTについて

- REINVENTはアストラゼネカ社が開発している強化学習をベースとした構造生成プログラムです。詳細はReadmeにリンクがあるので原著論文をご覧ください。

- 本チュートリアルではEGFR阻害剤を転移学習したモデルを作成し、転移学習前後の構造を見比べてみます。さらに転移学習後の構造を利用しVinaでDockingを行います。

- REINVENT4はTOMLファイルを準備しておくことで非常に簡単にプログラムを実行できます。TOMLファイルについてはREADMEを参考に事前に設定してくることを推奨します。

### FBP ref
- https://github.com/flowbased/flow-based.org